# Prepare data

All trajectories are stored in a Google Cloud Storage bucket. We want to be able to load and filter all trajectories easily.  To this end, we load all the datasets (lazily), filter them to different parameters (starting MPA, depth, stokes drift), and store a Pandas dataframe with virtual sub-datasets for each combination of the parameters.  This Pandas dataframe will be pickled for later re-use.

In [1]:
# parameters
dataset_version = "v2019.09.11.2"
bucket_stokes = f"pangeo-parcels/med_sea_connectivity_{dataset_version}/traj_data_with_stokes.zarr"
bucket_nostokes = f"pangeo-parcels/med_sea_connectivity_{dataset_version}/traj_data_without_stokes.zarr"

filter_warnings = "ignore"  # No warnings will bother you.  Change for debugging.

## Load all modules and spin up a Dask cluster

In [2]:
%matplotlib inline
from dask import array as da
import numpy as np
import xarray as xr
from gcsfs.mapping import GCSMap
from xhistogram.xarray import histogram as xhist
from matplotlib import pyplot as plt
import pandas as pd
from dask import delayed

In [3]:
from dask.distributed import Client, progress

from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=8)
cluster.adapt(minimum=8, maximum=60, wait_count=15)

client = Client(cluster)
client

/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://10.32.60.5:41427 Dashboard: /user/0000-0003-1951-8494/proxy/40099/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


** ☝️ Don't forget to click the link above to view the scheduler dashboard! **

## Open datasets

In [4]:
def open_dataset(bucket, restrict_to_MPA=None, restrict_to_z=None):
    # load data
    gcsmap = GCSMap(bucket)
    ds = xr.open_zarr(gcsmap, decode_cf=False)
    
    # get info on starting region and make it an easy-to-look-up coord
    initial_MPA = ds.MPA.isel(obs=0).squeeze()
    ds.coords["initial_MPA"] = initial_MPA
     
    # add mask that is False after land contact
    ds["before_land_contact"] = ((ds.land == 0).cumprod("obs") == 1)
      
    return ds

In [5]:
ds_stokes = open_dataset(bucket_stokes)
ds_nostokes = open_dataset(bucket_nostokes)

## Simplify

We know a few things about our data that make it easier to deal with them:

- No vertical migration.  Hence, initial depth of a particle is valid for all times.

- All time steps are the same. Hence, we can easily build a relative time axis that is valid for all particles.

In [6]:
def apply_assumptions(ds):
    """Applies simplifications to the dataset that are valid for the 
    specific set of experiments we're dealing with here.
    
    Be careful when applying these to new experiments, because
    they might not apply.
    """
    # We assume no vertical migration and hence
    # make (non-changing) depth level an easy to look up coord
    z = ds.z.isel(obs=0).squeeze()
    ds["z"] = z
    ds.coords["z"] = ds.z
    
    # We assume that all time steps are equal
    # and that the time axis is measured in seconds
    # since some reference period
    time_axis = ds.reset_coords(["z", "initial_MPA"]).time.isel(traj=0).squeeze()
    time_axis -= time_axis.isel(obs=0).squeeze()
    time_axis.attrs["units"] = "seconds since start of particle"
    ds.coords["time_axis"] = time_axis
    
    return ds

In [7]:
ds_stokes = apply_assumptions(ds_stokes)
ds_nostokes = apply_assumptions(ds_nostokes)

## Handle land contact

In [8]:
def mask_after_land_contact(ds):
    return ds.where(ds.before_land_contact)

In [9]:
ds_stokes = mask_after_land_contact(ds_stokes)
ds_nostokes = mask_after_land_contact(ds_nostokes)

## Load coordinates for quicker access

So far, we did only the bare minimum of information (data types, variable names, number of time steps, ...) but did not load any of the data.  We want to continue to do so for the bulk of the data, but get coordinates and the like now.

In [10]:
def persist_coords(ds, retries=40):
    """Will load coordinate data to the cluster."""
    ds["z"] = ds["z"].persist(retries=retries)
    ds["initial_MPA"] = ds["initial_MPA"].persist(retries=retries)
    ds["time_axis"] = ds["time_axis"].persist(retries=retries)
    return ds

In [11]:
def compute_coords(ds, retries=40):
    """Will load coordinate data to the front end."""
    ds["z"] = ds["z"].compute(retries=retries)
    ds["initial_MPA"] = ds["initial_MPA"].compute(retries=retries)
    ds["time_axis"] = ds["time_axis"].compute(retries=retries)
    return ds

In [12]:
ds_stokes = persist_coords(ds_stokes)
ds_nostokes = persist_coords(ds_nostokes)

In [13]:
ds_stokes = compute_coords(ds_stokes)
ds_nostokes = compute_coords(ds_nostokes)

In [14]:
ds_stokes

<xarray.Dataset>
Dimensions:              (obs: 962, traj: 2625480)
Coordinates:
    z                    (traj) float32 1.0182366 1.0182366 ... 1.0182366
    initial_MPA          (traj) float32 1.0 1.0 1.0 1.0 1.0 ... 9.0 9.0 9.0 9.0
    time_axis            (obs) float64 0.0 3.6e+03 ... 3.456e+06 3.456e+06
Dimensions without coordinates: obs, traj
Data variables:
    MPA                  (traj, obs) float32 dask.array<shape=(2625480, 962), chunksize=(100000, 962)>
    distance             (traj, obs) float32 dask.array<shape=(2625480, 962), chunksize=(100000, 962)>
    land                 (traj, obs) float32 dask.array<shape=(2625480, 962), chunksize=(100000, 962)>
    lat                  (traj, obs) float32 dask.array<shape=(2625480, 962), chunksize=(100000, 962)>
    lon                  (traj, obs) float32 dask.array<shape=(2625480, 962), chunksize=(100000, 962)>
    temp                 (traj, obs) float32 dask.array<shape=(2625480, 962), chunksize=(100000, 962)>
    time      

In [15]:
ds_nostokes

<xarray.Dataset>
Dimensions:              (obs: 962, traj: 13188600)
Coordinates:
    z                    (traj) float32 1.0182366 1.0182366 ... 10.536604
    initial_MPA          (traj) float32 nan nan nan nan nan ... 9.0 9.0 9.0 9.0
    time_axis            (obs) float64 nan nan nan nan nan ... nan nan nan nan
Dimensions without coordinates: obs, traj
Data variables:
    MPA                  (traj, obs) float32 dask.array<shape=(13188600, 962), chunksize=(100000, 962)>
    distance             (traj, obs) float32 dask.array<shape=(13188600, 962), chunksize=(100000, 962)>
    land                 (traj, obs) float32 dask.array<shape=(13188600, 962), chunksize=(100000, 962)>
    lat                  (traj, obs) float32 dask.array<shape=(13188600, 962), chunksize=(100000, 962)>
    lon                  (traj, obs) float32 dask.array<shape=(13188600, 962), chunksize=(100000, 962)>
    temp                 (traj, obs) float32 dask.array<shape=(13188600, 962), chunksize=(100000, 962)>
   

In [16]:
def get_z_values(ds):
    """Load unique z-values to the front end.
    
    This triggers a computation across all of the z-level data.
    """
    z_values = da.unique(ds.z.data).compute(retries=40)
    z_values = z_values[~np.isnan(z_values)]
    return z_values

In [17]:
z_values = get_z_values(ds_nostokes)

In [18]:
print(z_values)

[ 1.0182366  3.1657474  5.4649634  7.9203773 10.536604 ]


## Filter data

We want to quickly select:
- stokes drift on or off
- MPA a trajectory started from
- z-level

In [19]:
def restrict_to(ds, MPA=None, z=None):
    traj_indices = xr.full_like(ds.initial_MPA, True, dtype="bool")
    
    if MPA is not None:
        traj_indices = traj_indices & (ds.initial_MPA == MPA)
    
    if z is not None:
        traj_indices = traj_indices & (ds.z == z)
        
    ds = ds.isel(traj=traj_indices)
    
    return ds

In [20]:
from collections import OrderedDict

In [21]:
def wrap_in_dataframe(ds, stokes=True, num_levels=1, num_mpas=9):
    if num_levels is not None:
        data = pd.DataFrame(
            (
                OrderedDict(
                    {
                        "stokes": stokes, "MPA": MPA, "k": k,
                        "data": restrict_to(ds, MPA=MPA, z=z_values[k])
                    }
                )
                for MPA in range(1, 1 + num_mpas)
                for k in range(num_levels)
            )
        )
    else:
        data = pd.DataFrame(
            (
                OrderedDict(
                    {
                        "stokes": stokes, "MPA": MPA, "k": -1,
                        "data": restrict_to(ds, MPA=MPA, z=None)
                    }
                )
                for MPA in range(1, 1 + num_mpas)
            )
        )
    return data

The following will trigger computations.

In [22]:
# quick-access dataframe for stokes drift data at surface
data = wrap_in_dataframe(ds_stokes, stokes=True, num_levels=1, num_mpas=9)

# add non-stokes data per level
data = data.append(
    wrap_in_dataframe(ds_nostokes, stokes=False, num_levels=len(z_values), num_mpas=9),
    ignore_index=True
)

# add non-stokes data without distinguishing levels
data = data.append(
    wrap_in_dataframe(ds_nostokes, stokes=False, num_levels=None, num_mpas=9),
    ignore_index=True
)

In [23]:
data

,stokes,MPA,k,data
0,True,1,0,"[MPA, distance, land, lat, lon, temp, time, be..."
1,True,2,0,"[MPA, distance, land, lat, lon, temp, time, be..."
2,True,3,0,"[MPA, distance, land, lat, lon, temp, time, be..."
3,True,4,0,"[MPA, distance, land, lat, lon, temp, time, be..."
4,True,5,0,"[MPA, distance, land, lat, lon, temp, time, be..."
5,True,6,0,"[MPA, distance, land, lat, lon, temp, time, be..."
6,True,7,0,"[MPA, distance, land, lat, lon, temp, time, be..."
7,True,8,0,"[MPA, distance, land, lat, lon, temp, time, be..."
8,True,9,0,"[MPA, distance, land, lat, lon, temp, time, be..."
9,False,1,0,"[MPA, distance, land, lat, lon, temp, time, be..."


## Create thinned out data

We're not sure if we need all the statistics.  Create sub-sampled datasets that only have 1%, 5%, and 10% of the data.  Sub-sampling is done randomly.

In [24]:
def get_thinned_data(ds, percent=50, seed=None):
    """Return dataset thinned to a percentage by randomly picking trajectories."""
    if seed is not None:
        np.random.seed(seed)
    traj_indices = (np.random.uniform(0, 1, size=ds.z.shape) < (percent / 100.0))
    ds = ds.isel(traj=traj_indices)
    return ds

In [25]:
for perc in [1, 5, 10]:
    data[f"thinned_data_{perc:03d}_percent"] = data["data"].apply(lambda ds: get_thinned_data(ds, percent=perc))

In [26]:
data

,stokes,MPA,k,data,thinned_data_001_percent,thinned_data_005_percent,thinned_data_010_percent
0,True,1,0,"[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be..."
1,True,2,0,"[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be..."
2,True,3,0,"[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be..."
3,True,4,0,"[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be..."
4,True,5,0,"[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be..."
5,True,6,0,"[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be..."
6,True,7,0,"[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be..."
7,True,8,0,"[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be..."
8,True,9,0,"[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be..."
9,False,1,0,"[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be...","[MPA, distance, land, lat, lon, temp, time, be..."


## Make it easy to index

In [27]:
data = data.set_index(keys=["stokes", "MPA", "k"])

## Check data volumes

In [28]:
def get_total_size(series):
    return series.apply(lambda dobj: dobj.nbytes).sum()

In [29]:
print("All data (doubly counting non-stokes data):", get_total_size(data["data"]) / 1e9, "GB")
print("All thinned data (10%):", get_total_size(data["thinned_data_010_percent"]) / 1e9, "GB")
print("All thinned data (5%):", get_total_size(data["thinned_data_005_percent"]) / 1e9, "GB")
print("All thinned data (1%):", get_total_size(data["thinned_data_001_percent"]) / 1e9, "GB")

All data (doubly counting non-stokes data): 1027.481174776 GB
All thinned data (10%): 102.714949968 GB
All thinned data (5%): 51.434577944 GB
All thinned data (1%): 10.276203528 GB


In [30]:
import warnings
warnings.filterwarnings(filter_warnings)

## Store dataframe for later re-use.  Then re-load to check.

In [31]:
import cloudpickle

In [32]:
!mkdir -p intermediate_data

In [33]:
with open("intermediate_data/all_traj_dataframe.pickle", mode="wb") as f:
    cloudpickle.dump(data, f)

In [34]:
with open("intermediate_data/all_traj_dataframe.pickle", mode="rb") as f:
    data = cloudpickle.load(f)

# Technical documentation

Lists the whole working environment.

In [35]:
%pip list

Package                 Version    
----------------------- -----------
absl-py                 0.8.0      
adal                    1.2.2      
affine                  2.2.2      
aiohttp                 3.5.4      
alembic                 1.1.0      
altair                  3.2.0      
antlr4-python3-runtime  4.7.2      
appdirs                 1.4.3      
asciitree               0.3.3      
asn1crypto              0.24.0     
astor                   0.7.1      
async-generator         1.10       
async-timeout           3.0.1      
attrdict                2.0.1      
attrs                   19.1.0     
backcall                0.1.0      
basemap                 1.2.1      
beautifulsoup4          4.8.0      
bleach                  3.1.0      
blinker                 1.4        
blosc                   1.8.1      
bokeh                   1.3.4      
boto3                   1.9.221    
botocore                1.12.221   
branca                  0.3.1      
cached-property         1.5.

In [36]:
%conda list --explicit

# This file may be used to create an environment using:
# $ conda create --name <env> --file <this file>
# platform: linux-64
@EXPLICIT
https://repo.anaconda.com/pkgs/main/linux-64/_libgcc_mutex-0.1-main.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/ca-certificates-2019.6.16-hecc5488_0.tar.bz2
https://repo.anaconda.com/pkgs/main/linux-64/libgfortran-ng-7.3.0-hdf63c60_0.conda
https://repo.anaconda.com/pkgs/main/linux-64/libstdcxx-ng-9.1.0-hdf63c60_0.conda
https://conda.anaconda.org/conda-forge/linux-64/mpi-1.0-mpich.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/pandoc-2.7.3-0.tar.bz2
https://conda.anaconda.org/conda-forge/noarch/poppler-data-0.4.9-1.tar.bz2
https://repo.anaconda.com/pkgs/main/linux-64/toolchain-2.4.0-0.tar.bz2
https://repo.anaconda.com/pkgs/main/linux-64/libgcc-ng-9.1.0-hdf63c60_0.conda
https://conda.anaconda.org/conda-forge/linux-64/tbb-2018.0.5-h2d50403_0.tar.bz2
https://repo.anaconda.com/pkgs/main/linux-64/toolchain_c_linux-64-2.4.0-0.tar.bz2
http